In [1]:
from __future__ import print_function
from ortools.graph import pywrapgraph

In [11]:
import networkx as nx

In [623]:
import seaborn as sns
import matplotlib as mplt
import matplotlib.pyplot as plt

%matplotlib inline

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [573]:
def strftime(epoch_v):
    return time.strftime("%Y-%m-%d %H:%M:%S %Z", time.localtime(float(epoch_v)))

In [706]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import geopandas

In [712]:
import requests
import pandas_bokeh
pandas_bokeh.output_notebook()

Loading BokehJS ...

### Ready-mix optimization

In [186]:
from geopy.distance import geodesic
import time

#### geo_map maps plant and JobSiteCodes to their lat/lon centroids

In [699]:
geo_map = None
with open("geo_map.json", "rt") as in_file:
    geo_map = json.loads(in_file.read())

#### Input JSON config data (plants and deliveries)

In [678]:
import json
info_map = {}

date_pick = [
    "2019-08-12",
    "2019-08-13",
    "2019-08-14",
    "2019-08-15",
    "2019-08-16"
]

for date_v in date_pick:
    print(date_v)
    with open("info_{}.json".format(date_v), "rt") as in_file:
        info = json.loads(in_file.read())
        info_map[date_v] = info

2019-08-12
2019-08-13
2019-08-14
2019-08-15
2019-08-16


In [324]:
ratio_duration_at_client = 0.432

class GraphAdapter:
    
    def __init__(self):
        self._G = nx.DiGraph()
        self._node_index = 0
        self._nodes_m = {}
        
    def add_node(self, key, label, attr=None):
        self._nodes_m[key] = self._node_index
        self._G.add_node(self._node_index, label=label)
        if attr is not None:
            for key, value in attr.items():
                self._G.nodes[self._node_index][key] = value
        ret_index = self._node_index
        self._node_index += 1
        return ret_index
    
    def get_node(self, key):
        return self._nodes_m[key]

    def add_edge(self, nodeFrom, nodeTo, capacity, cost, attr=None):
        self._G.add_edge(nodeFrom, nodeTo)
        self._G[nodeFrom][nodeTo]["capacity"] = capacity
        self._G[nodeFrom][nodeTo]["cost"] = cost
        if attr is not None:
            for key, value in attr.items():
                self._G[nodeFrom][nodeTo][key] = value
        #edges["{},{}".format(nodeFrom, nodeTo)] = (capacity, cost)
        
    def getG(self):
        return self._G
    
    def getNodeMap(self):
        return self._nodes_m


#### Create optimization graph

In [682]:


def greatCircleDistance(x, y):
    lat1, lon1 = x[1], x[0]
    lat2, lon2 = y[1], y[0]
    return geodesic((lat1, lon1), (lat2, lon2)).meters
    
def compute_driving_cost(plant_drive_model, client_info):
    A = plant_drive_model["A"]
    b = plant_drive_model["b"]
    plant_coordinates = plant_drive_model["centroid"]
    client_coordinates = client_info["centroid"]
    PlantJobDistance = greatCircleDistance(plant_coordinates, client_coordinates)
    fix_velocity = 30*0.277778 # m/s
    duration_fix = PlantJobDistance/fix_velocity
    
    estimated_duration_all = A*PlantJobDistance + b
    estimated_duration_at_client = ratio_duration_at_client * estimated_duration_all
    estimated_duration_to_client = (estimated_duration_all - estimated_duration_at_client)/2.0
    estimated_duration_to_client_sec = estimated_duration_to_client * 60.0
    #return PlantJobDistance, estimated_duration_to_client_sec
    return PlantJobDistance, duration_fix
    
def create_optimization_graph(info, time_interval=3600):

    #info["plants"] # plant_id, start_time, end_time, load_time, time_interval, truck_capacity
    #info["deliveries"] # client_id, arrival_time, pour_duration, revenue
    #info["cost_to_client"] # travel cost from plant_id to client_id
    #info["cost_from_client"] # travel cost from client_id to plant_id
        
    g = GraphAdapter()
        
    #create supersource node
    key = "super_source::"
    g.add_node(key, label="super_source")
    superSource = g.get_node(key)
    #create supersink node
    key = "super_sink::"
    g.add_node(key, label="super_sink")
    superSink = g.get_node(key)

    #loop through deliveries
    client_exits = {}
    for _, item in info["deliveries"].items():
        #client arrival
        key = "client_arrival:{}:{}".format(item["client_id"], item["arrival_time"])
        node1 = g.add_node(key, label="client_arrival", attr={
            "client_id": item["client_id"], 
            "arrival_time": item["arrival_time"],
            "pour_duration": item["pour_duration"]
        })
        #client exit
        exit_time = item["arrival_time"] + item["pour_duration"]
        key = "client_exit:{}:{}".format(item["client_id"], exit_time)
        node2 = g.add_node(key, label="client_exit", attr={
            "client_id": item["client_id"],
            "exit_time": exit_time,
            "pour_duration": item["pour_duration"], 
            "arrival_time": item["arrival_time"]
        })
        #add edge between client delivery and exist with the revenue as positive cost
        g.add_edge(node1, node2, 1, -1 * item["revenue"] * 10000)
        
        client_exits[key] = {
            "client_id": item["client_id"],
            "exit_time": exit_time
        }
        
    last_time_nodes = []
    #loop through the plants
    for _, plant in info["plants"].items():
        prevTimeNode = None
        
        plant_drive_model = info["driving_model_data"]["plant_data"][plant["plant_id"]]
        
        latest_time_node = None
        #for t in range(plant["start_time"], plant["end_time"], plant["time_interval"]):
        for t in range(plant["start_time"], plant["end_time"], time_interval):
            #plant+time node
            key = "plant_time:{}:{}".format(plant["plant_id"],t)
            node1 = g.add_node(key, label="plant_time", attr={
                "plant_id": plant["plant_id"],
                "time": t
            })
            latest_time_node = node1
            #plant+time-load node
            key = key = "plant_time_load:{}:{}".format(plant["plant_id"],t)
            loadNode = g.add_node(key, label="plant_time_load", attr={
                "plant_id": plant["plant_id"],
                "time": t
            })
            #add edge between the plant-time node and the plant+time-load node
            g.add_edge(node1, loadNode, 1, time_interval) # putting fixed five finutes as we don't have this time
            
            if prevTimeNode is not None:
                #add edge from prevous time node to current time node
                g.add_edge(prevTimeNode, node1, 100000, 1) # 
            else:
                #then this is the first time node for the plant, so need to connect it to the super source
                g.add_edge(superSource, node1, plant["truck_capacity"], 0)
                
            prevTimeNode = node1
            
            #loop through the deliveries
            for _, c in info["deliveries"].items():
                # if the time of the load node + travel time <= required arrival, then create the connection
                distance, travel_cost = compute_driving_cost(plant_drive_model, 
                                     info["driving_model_data"]["client_data"][c["client_id"]])
                if ((t + travel_cost) <= c["arrival_time"]) and \
                   ((t + 90*60) >= c["arrival_time"]):
                    key = "client_arrival:{}:{}".format(c["client_id"], c["arrival_time"])
                    clientArrival = g.get_node(key)
                    #actual_cost = (c["arrival_time"] - t)
                    actual_cost = (c["arrival_time"] - t)
                    g.add_edge(loadNode, clientArrival, 1, actual_cost, attr={
                        "distance": distance,
                        "duration": actual_cost,
                        "estimated_duration": travel_cost
                    })
                    
        last_time_nodes.append((latest_time_node, plant["truck_capacity"]))
                    
    #loop through the client exits and find the closest plant time to return to for each plant
    for exit_key, exit_info in client_exits.items():
        for _, plant in info["plants"].items():
            plant_drive_model = info["driving_model_data"]["plant_data"][plant["plant_id"]]
            distance, travel_cost = compute_driving_cost(plant_drive_model, 
                             info["driving_model_data"]["client_data"][exit_info["client_id"]])
            for t in range(plant["start_time"], plant["end_time"], time_interval):
                if ((exit_info["exit_time"] + travel_cost) <= t):
                    # add edge between exit time and 
                    exitNode = g.get_node(exit_key)
                    actual_cost = (t - exit_info["exit_time"])
                    key = "plant_time:{}:{}".format(plant["plant_id"],t)
                    plant_time_node = g.get_node(key)
                    g.add_edge(exitNode, plant_time_node, 1, actual_cost, attr={
                        "distance": distance,
                        "duration": actual_cost,
                        "estimated_duration": travel_cost
                    })
                    break
                    
    for time_node, truck_capacity in last_time_nodes:
        g.add_edge(time_node, superSink, truck_capacity, 0)
    
    return g
    

##### Solve optimization problem 

In [683]:
def main(g, total_number_trucks=47):
    """MinCostFlow for ready-mix concrete route optimization."""
    
    # Define four parallel arrays: start_nodes, end_nodes, capacities, and unit costs
    # between each pair. For instance, the arc from node 0 to node 1 has a
    # capacity of 15 and a unit cost of 4.
    
    start_nodes = []  
    end_nodes   = []
    capacities  = []
    unit_costs  = []
    
    for edge in g.getG().edges:
        start_nodes.append(edge[0])
        end_nodes.append(edge[1])
        capacities.append(g.getG().edges[edge[0], edge[1]]['capacity'])
        unit_costs.append(int(g.getG().edges[edge[0], edge[1]]['cost']))
    
    # Define an array of supplies at each node.

    supplies = [total_number_trucks, -total_number_trucks]
    for i in range(2, g.getG().number_of_nodes()):
        supplies.append(0)
        
    print(len(start_nodes))
    print(len(end_nodes))
    print(len(capacities))
    print(len(unit_costs))
    print(len(supplies))
    
    # Instantiate a SimpleMinCostFlow solver.
    min_cost_flow = pywrapgraph.SimpleMinCostFlow()
    
    # Add each arc.
    for i in range(0, len(start_nodes)):
        min_cost_flow.AddArcWithCapacityAndUnitCost(start_nodes[i], end_nodes[i],
                                                capacities[i], unit_costs[i])
    
    # Add node supplies.
    for i in range(0, len(supplies)):
        min_cost_flow.SetNodeSupply(i, supplies[i])
        
    # Find the minimum cost flow between node 0 and node 4.
    xx = min_cost_flow.Solve()
    print("min_cost_flow.Solve() = {}".format(xx))
    plant_df = None
    if xx == min_cost_flow.OPTIMAL:
        print('Flow cost of the min flow cost', min_cost_flow.MaximumFlow())
        print('Minimum cost:', min_cost_flow.OptimalCost())
        print('')
        print('  Arc    Flow / Capacity  Cost')
        total_distance = 0
        total_duration = 0
        total_duration_est = 0
        total_distance_client = 0
        total_duration_client = 0
        total_duration_est_client = 0
        number_client_exists = 0
        delivery_info = {}
        plant_info = {}
        plant_stats = {}
        client_id = '66646736'
        for i in range(min_cost_flow.NumArcs()):
            cost = min_cost_flow.Flow(i) * min_cost_flow.UnitCost(i)
            if min_cost_flow.Flow(i) > 0 and False:
                print(',%1s -> %1s,   %3s  / %3s       %3s' % (
                  min_cost_flow.Tail(i),
                  min_cost_flow.Head(i),
                  min_cost_flow.Flow(i),
                  min_cost_flow.Capacity(i),
                cost))
            
            if min_cost_flow.Flow(i) > 0:
                src = min_cost_flow.Tail(i)
                srcn = g.getG().nodes[src]
                dst = min_cost_flow.Head(i)
                dstn = g.getG().nodes[dst]
                if srcn["label"].startswith("plant_time_load"):
                    ll = plant_info.setdefault(srcn["plant_id"], [])
                    if dstn["label"] != "super_sink":
                        arrival_time = strftime(dstn["arrival_time"])
                        edge = g.getG().edges[src, dst]
                        ll.append({
                            "plant_time": strftime(srcn["time"]),
                            "exp_time_back": strftime(srcn["time"] + 2*edge["duration"] + dstn["pour_duration"]),
                            "label": "arrival", 
                            "client_id": dstn["client_id"], 
                            "arrival_time": arrival_time,
                            "distance": edge["distance"],
                            "estimated_duration": edge["estimated_duration"],
                            "duration": edge["duration"]
                            
                        })
                        m = plant_stats.setdefault(srcn["plant_id"], {})
                        m.setdefault("send", 0)
                        m["send"] += 1
                    else:
                        ll.append(dstn)
                if dstn["label"] == "plant_time" and srcn["label"] != "plant_time":
                    ll = plant_info.setdefault(dstn["plant_id"], [])
                    if srcn["label"] != "super_source":
                        exit_time = strftime(srcn["exit_time"])
                        edge = g.getG().edges[src, dst]
                        ll.append({
                            "plant_time": strftime(dstn["time"]),
                            "label": "exit", 
                            "client_id": srcn["client_id"], 
                            "exit_time": exit_time,
                            "distance": edge["distance"],
                            "estimated_duration": edge["estimated_duration"],
                            "duration": edge["duration"]
                        })
                        m = plant_stats.setdefault(dstn["plant_id"], {})
                        m.setdefault("recv", 0)
                        m["recv"] += 1
                    else:
                        ll.append(srcn)
                if srcn["label"].startswith("client_exit"):
                    # then from client to plant
                    dist = g.getG().edges[src, dst]["distance"]
                    dur = g.getG().edges[src, dst]["duration"]
                    est_dur = g.getG().edges[src, dst]["estimated_duration"]
                    total_distance += dist
                    total_duration += dur
                    total_duration_est += est_dur
                    #print(dist)
                    number_client_exists += 1
                    m = delivery_info.setdefault("{}_{}".format(src, srcn["client_id"]), {})
                    arrival_time = strftime(srcn["arrival_time"])
                    ll = m.setdefault(arrival_time, [])
                    ll.append(("TO:",dstn["plant_id"], dist, dur/60.0, est_dur/60.0))
                    if srcn["client_id"] == client_id:
                        total_distance_client += dist
                        total_duration_client += dur
                        total_duration_est_client += est_dur
                if dstn["label"].startswith("client_arrival"):
                    # then from plant to client
                    dist = g.getG().edges[src, dst]["distance"]
                    dur = g.getG().edges[src, dst]["duration"]
                    est_dur = g.getG().edges[src, dst]["estimated_duration"]
                    total_distance += dist
                    total_duration += dur
                    total_duration_est += est_dur
                    m = delivery_info.setdefault("{}_{}".format(dst+1, dstn["client_id"]), {})
                    arrival_time = strftime(dstn["arrival_time"])
                    ll = m.setdefault(arrival_time, [])
                    ll.append(("FROM:",srcn["plant_id"], dist, dur/60.0, est_dur/60.0))
                    if dstn["client_id"] == client_id:
                        total_distance_client += dist
                        total_duration_client += dur
                        total_duration_est_client += est_dur
                    
        print("total_distance: {}".format(total_distance))
        print("total_duration: {}".format(total_duration))
        print("total_duration_est: {}".format(total_duration_est))
        print("total_distance_client: {}".format(total_distance_client))
        print("total_duration_client: {}".format(total_duration_client))
        print("total_duration_est_client: {}".format(total_duration_est_client))
        print("number_client_exists: {}".format(number_client_exists))
        for cid, info in delivery_info.items():
            if client_id in cid:
                print("client: {}".format(cid))
                for tt, info2 in info.items():
                    print("\tarrival: {}:".format(tt))
                    for z in info2[::-1]:
                        print("\t\t{}".format(z))
        for pid, info in plant_stats.items():
            print("plant: {}".format(pid))
            print("\tsend: {}".format(info["send"]))
            print("\trecv: {}".format(info["recv"]))
    
        data = []
        for pid, item_list in plant_info.items():
            print("plant: {}".format(pid))
            sorted_list = sorted([x for x in item_list if x["label"] == "exit" or x["label"] == "arrival"], key=lambda x: x["arrival_time"] if "arrival_time" in x else x["exit_time"])
            for item in sorted_list:
                #print("\t{}".format(item))
                record = [pid, 
                          item["plant_time"],
                          item["exp_time_back"] if "arrival_time" in item else None,
                          "arrival" if "arrival_time" in item else "exit",
                          item["client_id"], 
                          item["arrival_time"] if "arrival_time" in item else item["exit_time"],
                          item["distance"]/1000.0,
                          item["estimated_duration"]/60.0,
                          item["duration"]/60.0
                         ]
                data.append(record)
        plant_df = pd.DataFrame(data, columns=[
            "Plant", "PTime", "ExpTimeBack", "OType", "Client", "Time", "Dist(km)", "EstDur(m)", "Dur(m)"
        ])
    else:
        print('There was an issue with the min cost flow input.')
    return plant_df

In [698]:
plant_stats_map = {}
g_map = {}
for date_v, info in info_map.items():
    print("Date: {}".format(date_v))
    print()
    g = create_optimization_graph(info, time_interval=300)
    g_map[date_v] = g
    G = g.getG()
    
    total_number_trucks = 0
    num_deliveries = len(info_map[date_v]["deliveries"].keys())
    for k,v in info_map[date_v]["plants"].items():
        total_number_trucks += int(v["truck_capacity"])
    print("total_number_trucks = {}".format(total_number_trucks))
    print("num_deliveries = {}".format(num_deliveries))
    
    plant_df = main(g, total_number_trucks=total_number_trucks)
    plant_stats_map[date_v] = plant_df

Date: 2019-08-12

total_number_trucks = 25
num_deliveries = 32
2410
2410
2410
2410
1274
min_cost_flow.Solve() = 1
Flow cost of the min flow cost 25
Minimum cost: -977174292

  Arc    Flow / Capacity  Cost
total_distance: 315335.1156469573
total_duration: 47476.56767630577
total_duration_est: 37840.183605487975
total_distance_client: 0
total_duration_client: 0
total_duration_est_client: 0
number_client_exists: 31
plant: FB40
	send: 6
	recv: 6
plant: F011
	send: 8
	recv: 8
plant: FB25
	send: 12
	recv: 12
plant: FB58
	send: 5
	recv: 5
plant: F011
plant: FB25
plant: FB40
plant: FB58
Date: 2019-08-13

total_number_trucks = 41
num_deliveries = 82
4396
4396
4396
4396
1558
min_cost_flow.Solve() = 1
Flow cost of the min flow cost 41
Minimum cost: -2581617316

  Arc    Flow / Capacity  Cost
total_distance: 1173996.4504975292
total_duration: 167218.58179664612
total_duration_est: 140879.46135613447
total_distance_client: 0
total_duration_client: 0
total_duration_est_client: 0
number_client_exists